In [1]:
import pandas as pd


In [2]:
file_path = '../Data/amazon_meta.json'  
amazon_df = pd.read_json(file_path, lines=True, compression=None)

In [3]:
file_path = "all_titles_only.csv"
# Read the CSV file as-is
toysrecall_df = pd.read_csv(file_path, encoding='utf-8')





In [4]:


print("Combined recall records:", len(toysrecall_df))
print("Columns:", toysrecall_df.columns.tolist())
toysrecall_df.head(20)

Combined recall records: 709
Columns: ['Title']


,Title
0,Kids II Inc. Recalls Crib Toys Due to Choking ...
1,Discovery Toys Children's Toy Phone Recall
2,Kids Station Toys Recalls Little Tikes Toy Cel...
3,Pop Up Phone Recalled; Antenna Is Choking Hazard
4,TDT Toy Company Recalls Plastic Telephones Bec...
5,Toy Telephones Recalled by Durham
6,"Coynes, Inc. Recalls Musical Cordless Toy Tele..."
7,Toy Baby Phone Recalled by Vtech
8,Toy Phones Recalled for Choking Hazard
9,Evenflo Recalls Telephone Toys Due to Choking ...


In [5]:
print(toysrecall_df.columns)

Index(['Title'], dtype='object')


In [10]:
from rapidfuzz import fuzz, process

In [37]:
from rapidfuzz import fuzz
import pandas as pd
import re

def clean_recall_title(title: str) -> str:
    banned_words = [
        'recall', 'recalls', 'recalled','choking', 'choked', 'hazard', 'hazards', 'hazardous',
        'title', 'danger', 'due', 'to', 'for', 'announce', 'announced', 'announcement',
        'alert', 'alerts', 'warning', 'warnings', 'safety', 'unsafe', 'product', 'products',
        'item', 'items', 'may', 'cause', 'risk', 'risks', 'injury', 'injuries', 'harm',
        'damages', 'defect', 'defective', 'faulty', 'fault', 'issue', 'issues',
        'problem', 'problems', 'dangerous', 'dangerously'
    ]
    pattern = r'\b(?:' + '|'.join(banned_words) + r')\b'
    cleaned = re.sub(pattern, '', title, flags=re.IGNORECASE)
    cleaned = re.sub(r'\s+', ' ', cleaned)  # remove extra spaces
    return cleaned.strip()

def match_amazon_to_recalls(amazon_df, recall_df, score_threshold=90):
    amazon = amazon_df.copy()
    recalls = recall_df.copy()

    amazon['title'] = amazon['title'].astype(str).str.lower().str.strip()
    recalls['Title'] = recalls['Title'].astype(str).str.lower().str.strip()

    # Initialize columns
    amazon_df['is_match'] = 0
    amazon_df['recall_index'] = [[] for _ in range(len(amazon_df))]

    matched = {}

    for recall_title in recalls['Title'].dropna().unique():
        cleaned = clean_recall_title(recall_title)
        if not cleaned:
            continue

        mask = amazon['title'].apply(lambda x: fuzz.token_sort_ratio(x, cleaned) >= score_threshold)
        amazon_matches = amazon[mask]
        recall_rows = recalls[recalls['Title'] == recall_title]

        if not amazon_matches.empty:
            matched[recall_title] = (recall_rows.copy(), amazon_matches.copy())
            recall_idx = recall_rows.index[0]

            for idx in amazon_matches.index:
                amazon_df.at[idx, 'is_match'] = 1
                amazon_df.at[idx, 'recall_index'].append(2514 + recall_idx)

    print(f"✅ Found matches for {len(matched)} recall-title entries.")
    print(f"📦 Unique Amazon products matched: {amazon_df['is_match'].sum()}")

    return matched, amazon_df


In [ ]:
from rapidfuzz import fuzz
import pandas as pd
import re

def clean_recall_title(title: str) -> str:
    banned_words = ['recall', 'recalls', 'recalled','choking', 'choked', 'hazard', 'hazards', 'hazardous', 'title', 'danger', "due", "to", "for", "announce", "announced", "announcement", "alert", "alerts", "warning", "warnings", "safety", "unsafe", "product", "products", "item", "items", "may", "cause", "risk", "risks", "injury", "injuries", "harm", "damages", "defect", "defective", "faulty", "fault", "issue", "issues", "problem", "problems", "dangerous", "dangerously"]
    pattern = r'\b(?:' + '|'.join(banned_words) + r')\b'
    cleaned = re.sub(pattern, '', title, flags=re.IGNORECASE)
    return cleaned.strip()

def match_amazon_to_recalls(amazon_df, recall_df, score_threshold=90):
    amazon = amazon_df.copy()
    recalls = recall_df.copy()

    amazon['title'] = amazon['title'].astype(str).str.lower().str.strip()
    recalls['Title'] = recalls['Title'].astype(str).str.lower().str.strip()

    amazon_df['is_match'] = 0
    amazon_df['recall_index'] = None

    matched = {}

    for recall_title in recalls['Title'].dropna().unique():
        cleaned = clean_recall_title(recall_title)
        if not cleaned:
            continue

        mask = amazon['title'].apply(lambda x: fuzz.token_sort_ratio(x, cleaned) >= score_threshold)
        amazon_matches = amazon[mask]
        recall_rows = recalls[recalls['Title'] == recall_title]

        if not amazon_matches.empty:
            matched[recall_title] = (recall_rows.copy(), amazon_matches.copy())
            recall_idx = recall_rows.index[0]

            for idx in amazon_matches.index:
                amazon_df.at[idx, 'is_match'] = 1
                amazon_df.at[idx, 'recall_index'] = 2514 + recall_idx

    print(f"✅ Found matches for {len(matched)} recall-title entries.")
    print(f"📦 Unique Amazon products matched: {amazon_df['is_match'].sum()}")

    return matched, amazon_df


In [38]:
matched_results, updated_amazon_df = match_amazon_to_recalls(amazon_df, toysrecall_df, 80)

✅ Found matches for 16 recall-title entries.
📦 Unique Amazon products matched: 26


In [40]:
for recall_title, (recall_df, amazon_df_match) in matched_results.items():
    print(f"\n⚠️ Recall Title: {recall_title}")
    print("🛒 Matched Amazon Titles:")

    for _, row in amazon_df_match.iterrows():
        print(f"   - {row['title']}")



⚠️ Recall Title: toy mobile phones recalled for choking hazard
🛒 Matched Amazon Titles:
   - mobile phone
   - frozen toy mobile phone

⚠️ Recall Title: green toys recalls mini vehicles due to choking hazard
🛒 Matched Amazon Titles:
   - green toys mixer vehicle
   - green toys  mini vehicle, 4-pack

⚠️ Recall Title: infantino recalls toy activity trucks due to choking hazard
🛒 Matched Amazon Titles:
   - infantino activity toy set

⚠️ Recall Title: fisher-price recalls to repair little people builders' load 'n go wagons due to laceration hazard
🛒 Matched Amazon Titles:
   - fisher-price little people builders load 'n go wagon

⚠️ Recall Title: wooden toy alphabet blocks recalled
🛒 Matched Amazon Titles:
   - aleph bet wooden blocks
   - wooden alphabet blocks - wagon

⚠️ Recall Title: toys r us recalls imaginarium activity center due to choking hazard
🛒 Matched Amazon Titles:
   - imaginarium 5 sided activity center

⚠️ Recall Title: target recalls activity cart toys due to choking h

In [41]:
updated_amazon_df.to_csv('amazon_meta_with_recall_matches.csv', index=False, encoding='utf-8')
print("Updated Amazon DataFrame saved to 'amazon_meta_with_recall_matches.csv'.")

Updated Amazon DataFrame saved to 'amazon_meta_with_recall_matches.csv'.


In [42]:
print(updated_amazon_df.columns)

Index(['category', 'tech1', 'description', 'fit', 'title', 'also_buy', 'tech2',
       'brand', 'feature', 'rank', 'also_view', 'main_cat', 'similar_item',
       'date', 'price', 'asin', 'imageURL', 'imageURLHighRes', 'details',
       'is_match', 'recall_index'],
      dtype='object')
